In [1]:
# syft absolute
import syft as sy
from syft import UID

In [2]:
# we can generate a thin wrapper like action object but without the bloat
# we synthesize a custom type so that it acts like the real type for 99.9999% of cases

In [3]:
# stdlib
from typing import Any
from typing import Generic
from typing import TypeVar

T = TypeVar("T")


class SyftAutoboxMeta(type):
    def __call__(cls, value: T):
        wrapped_type = type(value)
        dynamic_class_name = f"SyftAutobox[{wrapped_type.__name__}]"

        # Dynamically create a new class that inherits from the wrapped type only
        DynamicWrapper = type(
            f"SyftAutobox{wrapped_type.__name__.capitalize()}", (wrapped_type,), {}
        )

        class Wrapped(DynamicWrapper):
            def __init__(self, value: Any, uid: UID | None = None):
                self._syft_value = value
                self._syft_uid = uid if uid is not None else UID()
                super(DynamicWrapper, self).__init__()

            def __getattribute__(self, name: str) -> Any:
                # Bypass certain attrs to prevent recursion issues
                if name.startswith("_syft"):
                    return object.__getattribute__(self, name)

                return getattr(self._syft_value, name)

            # these empty dunders are required
            def __repr__(self):
                return self.__repr__()

            def __str__(self):
                return self.__str__()

            def __dir__(self):
                return self.__dir__()

        Wrapped.__name__ = dynamic_class_name
        Wrapped.__qualname__ = dynamic_class_name
        return Wrapped(value)


class SyftAutobox(Generic[T], metaclass=SyftAutoboxMeta):
    pass

In [4]:
# a vanilla string
b = "test"

In [5]:
# a syft autoboxed string
a = SyftAutobox(value=b)

In [6]:
a

'test'

In [7]:
# type is nice and clean
type(a), type(b)

(__main__.SyftAutobox[str], str)

In [8]:
# passes isinstance check
assert isinstance(a, str) and isinstance(b, str)

In [9]:
# passes is subclass check
assert issubclass(type(a), str) and issubclass(type(b), str)

In [10]:
# we renamed one class so it looks nested
type(a).mro()

[__main__.SyftAutobox[str], __main__.SyftAutoboxStr, str, object]

In [11]:
type(b).mro()

[str, object]

In [12]:
# lets try numpy

In [13]:
# third party
import numpy as np

# Create a numpy array
arr = np.array([1, 2, 3, 4, 5])

# Define a function that expects a numpy array


def sum_array(arr: np.ndarray) -> int:
    return np.sum(arr)


# Test with a regular numpy array
print(sum_array(arr))  # Should print 15

15


In [14]:
arr

array([1, 2, 3, 4, 5])

In [15]:
# Wrap the numpy array
wrapped_arr = SyftAutobox(arr)

In [16]:
wrapped_arr

array([1, 2, 3, 4, 5])

In [17]:
sum_array(wrapped_arr)

15

In [18]:
# nice
wrapped_arr._syft_uid

<UID: 4b60b54ef7d544679257194980660b6d>

In [19]:
# third party
# lets try some stuff which might break with ActionObject
import numpy.ctypeslib as npct


def to_ctypes(arr: np.ndarray):
    return npct.as_ctypes(arr)

In [20]:
action_arr = sy.ActionObject.from_obj(arr)
action_arr


**Pointer**

array([1, 2, 3, 4, 5])


In [21]:
try:
    print(to_ctypes(action_arr))  # This will raise an exception
except TypeError as e:
    print(f"TypeError: {e}")

TypeError: Cannot interpret 'Pointer:
'<i8'' as a data type


In [22]:
# works for raw numpy
try:
    print(to_ctypes(arr))  # This will raise an exception
except TypeError as e:
    print(f"TypeError: {e}")

In [23]:
# works for autobox
try:
    print(to_ctypes(wrapped_arr))  # This will raise an exception
except TypeError as e:
    print(f"TypeError: {e}")

In [24]:
# okay now what if we made this wrapper type capable of intelligently handling Results?

In [25]:
# stdlib
from typing import Generic
from typing import TypeVar
from typing import Union

T = TypeVar("T")
E = TypeVar("E")


class Ok(Generic[T]):
    def __init__(self, value: T):
        self.value = value


class Err(Generic[E]):
    def __init__(self, error: E):
        self.error = error


Result = Union[Ok[T], Err[E]]

error_allowed_attrs = ["__class__"]


class SyftAutoboxMeta(type):
    def __call__(cls, value: Result[T, E] | T):
        if isinstance(value, Ok):
            wrapped_value = value.value
            is_error = False
        elif isinstance(value, Err):
            wrapped_value = value.error
            is_error = True
        else:
            wrapped_value = value
            is_error = False

        wrapped_type = type(wrapped_value)
        dynamic_class_name = f"SyftAutobox[{wrapped_type.__name__}]"

        # Dynamically create a new class that inherits from the wrapped type only
        DynamicWrapper = type(
            f"SyftAutobox{wrapped_type.__name__.capitalize()}", (wrapped_type,), {}
        )

        class Wrapped(DynamicWrapper):
            def __init__(self, value: Any, uid: UID | None = None):
                self._syft_value = value
                self._syft_uid = uid if uid is not None else UID()
                self._syft_is_error = is_error
                super(DynamicWrapper, self).__init__()

            def __getattribute__(self, name: str) -> Any:
                # Bypass certain attrs to prevent recursion issues
                if name.startswith("_syft"):
                    return object.__getattribute__(self, name)

                if self._syft_is_error and name not in error_allowed_attrs:
                    raise Exception(
                        f"Cannot access attribute '{name}' on an Err result: {self._syft_value}"
                    )

                return getattr(self._syft_value, name)

            # these empty dunders are required
            def __repr__(self):
                return self._syft_value.__repr__()

            def __str__(self):
                return self._syft_value.__str__()

            def __dir__(self):
                return self._syft_value.__dir__()

        Wrapped.__name__ = dynamic_class_name
        Wrapped.__qualname__ = dynamic_class_name
        return Wrapped(wrapped_value)


class SyftAutobox(Generic[T], metaclass=SyftAutoboxMeta):
    pass

In [26]:
# now we can freely return T or Result[T] and its the same
ok_value = SyftAutobox(Ok(42))
print(ok_value)  # Should print 42

42


In [27]:
type(ok_value)

__main__.SyftAutobox[int]

In [28]:
isinstance(ok_value, int)

True

In [29]:
# can perform addition, and can probably autowrap if we go and implement the __ like in actionobj
b = ok_value + 1
type(b)

int

In [30]:
# what about an error

In [31]:
# s = sy.SyftError(message="Some Error")

In [32]:
try:
    err_value = SyftAutobox(Err("Some Error"))
    print(err_value.upper())  # This should raise an exception
except Exception as e:
    print(e)  # Should raise an exception with the error message

Cannot access attribute 'upper' on an Err result: Some Error


In [33]:
# even though its got a string inside it should raise exceptions
err_value.upper()

Exception: Cannot access attribute 'upper' on an Err result: Some Error

In [34]:
# we can choose to raise exceptions when accessing error types however we like

In [35]:
try:
    err_value = SyftAutobox(Err(arr))
    print(err_value.dtype)  # This should raise an exception
except Exception as e:
    print(e)  # Should raise an exception with the error message

Cannot access attribute 'dtype' on an Err result: [1 2 3 4 5]


In [36]:
# rationale
# better than ActionObject already, as it passes more C code checks etc
# allows us to add some Result / Future functionality ONTOP of this type as desired
# allows a UID for linkage at all times
# solves issue of returning a Result[T, E] vs T raises Exception theres no difference when you autobox
# raises exceptions if we want it to

In [37]:
# TODO:
# autowrap results of infix operators
# raise exceptions on infix operators if its an Exception type
# make exception raising optional
# add is_ok, is_err, type methods to the Autobox
# add ._syft_ready, .is_ready
# Future
# __getattr__ -> self._syft_value
# check if its been longer than 1 second
# if name of getattr not in special ._syft_ready, .is_ready
# call server for update and then replace self._syft_value with real value

# obj.sum() <---

In [38]:
# method -> T, Exception
# method_async -> Result[T, Error]

In [39]:
# method -> Autobox[T, SyftErr[Message, Exception]]

# TBD
# method_async -> Autobox[Future[T, Exception], SyftErr[Message, Exception]]